# Keras example

Trains and evaluate a simple MLP on the Reuters newswire topic classification task.

In [2]:
import numpy as np

import keras
from keras.datasets import reuters
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.preprocessing.text import Tokenizer

# The following import and function call are the only additions to code required
# to automatically log metrics and parameters to MLflow.
import mlflow.keras

2023-05-30 23:10:40.332264: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-30 23:10:40.497546: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-30 23:10:40.497706: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
# mlflow server --backend-store-uri mlruns/ --default-artifact-root mlruns/ --host 0.0.0.0
remote_server_uri = "http://127.0.0.1:5000/" # set to your server URI
mlflow.set_tracking_uri(remote_server_uri)  # or set the MLFLOW_TRACKING_URI in the env

exp_name = "reuters_keras"
mlflow.set_experiment(exp_name)

2023/05/30 23:11:16 INFO mlflow.tracking.fluent: Experiment with name 'reuters_keras' does not exist. Creating a new experiment.


<Experiment: artifact_location='./mlruns/2', creation_time=1685468476802, experiment_id='2', last_update_time=1685468476802, lifecycle_stage='active', name='reuters_keras', tags={}>

In [4]:
mlflow.keras.autolog()

In [5]:
max_words = 1000
batch_size = 32
epochs = 5

mlflow.log_params({"max_words": max_words, "batch_size": batch_size, "epochs": epochs})

In [6]:
print('Loading data...')
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=max_words,
                                                         test_split=0.2)

print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

num_classes = np.max(y_train) + 1
print(num_classes, 'classes')

Loading data...
2110848/2110848 [==============================] - 0s 0us/step
8982 train sequences
2246 test sequences
46 classes


In [7]:
print('Vectorizing sequence data...')
tokenizer = Tokenizer(num_words=max_words)
x_train = tokenizer.sequences_to_matrix(x_train, mode='binary')
x_test = tokenizer.sequences_to_matrix(x_test, mode='binary')
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Vectorizing sequence data...
x_train shape: (8982, 1000)
x_test shape: (2246, 1000)


In [8]:
print('Convert class vector to binary class matrix '
      '(for use with categorical_crossentropy)')
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

Convert class vector to binary class matrix (for use with categorical_crossentropy)
y_train shape: (8982, 46)
y_test shape: (2246, 46)


In [9]:
print('Building model...')
model = Sequential()
model.add(Dense(512, input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Building model...


2023-05-30 23:11:31.195696: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-05-30 23:11:31.200359: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-05-30 23:11:31.200761: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (TIGER02266): /proc/driver/nvidia/version does not exist
2023-05-30 23:11:31.208954: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [10]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Epoch 1/5
253/253 [==============================] - 4s 10ms/step - loss: 1.4272 - accuracy: 0.6802 - val_loss: 1.0819 - val_accuracy: 0.7597
Epoch 2/5
253/253 [==============================] - 2s 8ms/step - loss: 0.7835 - accuracy: 0.8124 - val_loss: 0.9063 - val_accuracy: 0.8042
Epoch 3/5
253/253 [==============================] - 2s 9ms/step - loss: 0.5521 - accuracy: 0.8676 - val_loss: 0.8628 - val_accuracy: 0.8042
Epoch 4/5
253/253 [==============================] - 2s 8ms/step - loss: 0.4194 - accuracy: 0.8963 - val_loss: 0.8792 - val_accuracy: 0.8076
Epoch 5/5
1/1 [==============================] - 0s 173ms/step
INFO:tensorflow:Assets written to: /tmp/tmp2gf65y25/model/data/model/assets


2023/05/30 23:12:04 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/dharani/miniconda3/envs/face_detection/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils."


In [11]:
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
print('Test score:', score[0])
print('Test accuracy:', score[1])
mlflow.log_metrics({"test_accuarcy": score[1], "test_loss": score[0]})

71/71 [==============================] - 0s 4ms/step - loss: 0.9080 - accuracy: 0.7952
Test score: 0.9079836010932922
Test accuracy: 0.7951914668083191
